# 네이버 웹툰 크롤링


*   URL: https://m.comic.naver.com/webtoon/list?titleId=769209&page=1&sort=DESC
*   <화산귀환> 첫번째 목록 페이지로부터 링크 추출하여 각 회차 이미지 크롤링



## 라이브러리 임포트

In [ ]:
import requests
import os
from bs4 import BeautifulSoup as bs

## URL 설정 및 목록 페이지 response 받아오기

In [ ]:
url = f'https://m.comic.naver.com/webtoon/list?titleId=769209&page=1&sort=DESC'

res = requests.get(url)

## 받아온 response를 BeautifulSoup 객체로 변환

In [ ]:
html = res.text
html_sp = bs(html, 'html.parser')

## 회차의 제목 리스트 정렬하기

In [ ]:
episode_index_list = html_sp.select('span.name')

for idx, tag in enumerate(episode_index_list):
  episode_index_list[idx] = tag.string

episode_index_list

['81화',
 '80화',
 '79화',
 '78화',
 '77화',
 '76화',
 '75화',
 '74화',
 '73화',
 '1부 후기',
 '72화(1부 최종화)',
 '71화',
 '70화',
 '69화',
 '68화',
 '67화',
 '66화',
 '65화',
 '64화',
 '63화',
 '62화',
 '61화',
 '60화',
 '59화',
 '58화',
 '57화',
 '56화',
 '55화',
 '54화',
 '53화',
 '52화',
 '51화',
 '50화',
 '49화']

## 회차의 URL 리스트 정렬하기

In [ ]:
# 회차와 그 회차의 url 리스트로 뽑아내기
episode_url_tags = html_sp.select('ul.section_episode_list li.item a.link')

episode_url_list = []
for episode in episode_url_tags:
    episode_url_list.append(episode['href'])

# 위에 미리보기 4개가 있기 때문에 '#'만 출력된다. 지우기:
del episode_url_list[0:4]
episode_url_list

['/webtoon/detail?titleId=769209&no=78&week=wed&listSortOrder=DESC&listPage=1',
 '/webtoon/detail?titleId=769209&no=77&week=wed&listSortOrder=DESC&listPage=1',
 '/webtoon/detail?titleId=769209&no=76&week=wed&listSortOrder=DESC&listPage=1',
 '/webtoon/detail?titleId=769209&no=75&week=wed&listSortOrder=DESC&listPage=1',
 '/webtoon/detail?titleId=769209&no=74&week=wed&listSortOrder=DESC&listPage=1',
 '/webtoon/detail?titleId=769209&no=73&week=wed&listSortOrder=DESC&listPage=1',
 '/webtoon/detail?titleId=769209&no=72&week=wed&listSortOrder=DESC&listPage=1',
 '/webtoon/detail?titleId=769209&no=71&week=wed&listSortOrder=DESC&listPage=1',
 '/webtoon/detail?titleId=769209&no=70&week=wed&listSortOrder=DESC&listPage=1',
 '/webtoon/detail?titleId=769209&no=69&week=wed&listSortOrder=DESC&listPage=1',
 '/webtoon/detail?titleId=769209&no=68&week=wed&listSortOrder=DESC&listPage=1',
 '/webtoon/detail?titleId=769209&no=67&week=wed&listSortOrder=DESC&listPage=1',
 '/webtoon/detail?titleId=769209&no=66&w

## headers 설정

In [ ]:
# headers 설정
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
headers = {
    'User-Agent' : user_agent
}

## 회차의 URL을 인자로 받아 그 안의 이미지 링크를 리턴하는 함수 정의하기

In [47]:
def get_image_links(url):
  res_html = requests.get(url).text

  # bs 객체로 변환
  html_sp = bs(res_html, 'html.parser')

  # 이미지 태그 찾기
  image_tags = html_sp.select('p>img')

  # 이미지 태그에서 url 찾기
  image_urls = []

  for tag in image_tags:
    image_urls.append(tag['data-src'])

  return image_urls

## 이미지 저장

In [48]:
# 모든 회차의 이미지 크롤링하여 지정한 경로에 저장하기
# 저장한 경로에 들어갈 때, 상대 경로로 지정되어 있으므로 직접 절대 경로로 설정해주어야 한다.
import os
root_url = 'https://m.comic.naver.com/'

for ep_idx, url in enumerate(episode_url_list):
  dir_path = f'화산귀환/{episode_index_list[ep_idx]}'
  os.makedirs(dir_path, exist_ok = True)

  # 페이지 안의 이미지 링크들을 리턴해주는 함수
  episode_image_urls = get_image_links(root_url + url)

  for pg_idx, img_url in enumerate(episode_image_urls, 1):
    file_path = f'{dir_path}/{pg_idx:03d}.jpg'

    image_res = requests.get(img_url, headers = headers)
    assert image_res.status_code == 200, "서버 응답이 200이 아닙니다."

    # 저장
    with open(file_path, 'wb') as f:
      f.write(image_res.content)

KeyboardInterrupt: ignored